In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score, auc, accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample
df=pd.read_csv('D:\\GL\\ml-gl-backup-master\\ml-gl-backup-master\\Projects\\Project1\\adult.csv')

In [54]:
df.head()

,age,work class,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per -week,country,classification
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age                32561 non-null int64
work class         32561 non-null object
fnlwgt             32561 non-null int64
education          32561 non-null object
education-num      32561 non-null int64
marital-status     32561 non-null object
occupation         32561 non-null object
relationship       32561 non-null object
race               32561 non-null object
sex                32561 non-null object
capital-gain       32561 non-null int64
capital-loss       32561 non-null int64
hours-per -week    32561 non-null int64
country            32561 non-null object
classification     32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [102]:
df_copy=df.copy(deep=True)

In [103]:
df_copy['country'] = df['country'].replace(' ?',np.nan)

In [104]:
df_copy['work class'] = df['work class'].replace(' ?',np.nan)

In [105]:
df_copy['occupation'] = df['occupation'].replace(' ?',np.nan)

In [106]:
df_copy.isna().sum()

age                   0
work class         1836
fnlwgt                0
education             0
education-num         0
marital-status        0
occupation         1843
relationship          0
race                  0
sex                   0
capital-gain          0
capital-loss          0
hours-per -week       0
country             583
classification        0
dtype: int64

In [107]:
df_copy.shape

(32561, 15)

In [108]:
df_drop_copy=df_copy.copy(deep=True)

In [109]:
df_drop_copy=df_drop_copy.dropna()

In [110]:
df_drop_copy.shape

(30162, 15)

In [111]:
df_drop_copy['sex'] = df_drop_copy['sex'].map({' Male':1,' Female':0}).astype(int)

In [112]:
salary_map={' <=50K':1,' >50K':0}
df_drop_copy['classification']=df_drop_copy['classification'].map(salary_map).astype(int)

In [113]:
df_drop_copy['marital-status'] = df_drop_copy['marital-status'].replace([' Divorced',' Married-spouse-absent',' Never-married',' Separated',' Widowed'],'Single')
df_drop_copy['marital-status'] = df_drop_copy['marital-status'].replace([' Married-AF-spouse',' Married-civ-spouse'],'Couple')

In [114]:
df_drop_copy['marital-status'] = df_drop_copy['marital-status'].map({'Couple':0,'Single':1})

In [115]:
rel_map = {' Unmarried':0,' Wife':1,' Husband':2,' Not-in-family':3,' Own-child':4,' Other-relative':5}

df_drop_copy['relationship'] = df_drop_copy['relationship'].map(rel_map)

In [116]:
race_map={' White':0,' Amer-Indian-Eskimo':1,' Asian-Pac-Islander':2,' Black':3,' Other':4}


df_drop_copy['race']= df_drop_copy['race'].map(race_map)

In [117]:
df_drop_copy['work class'].value_counts()

 Private             22286
 Self-emp-not-inc     2499
 Local-gov            2067
 State-gov            1279
 Self-emp-inc         1074
 Federal-gov           943
 Without-pay            14
Name: work class, dtype: int64

In [118]:
def work_class(x):
    if x['work class'] == ' Federal-gov' or x['work class']== ' Local-gov' or x['work class']==' State-gov': 
        return 'govt'
    elif x['work class'] == ' Private':
        return 'private'
    elif x['work class'] == ' Self-emp-inc' or x['work class'] == ' Self-emp-not-inc': 
        return 'self_employed'
    else: 
        return 'without_pay'
    
    
df_drop_copy['employment_type']=df_drop_copy.apply(work_class, axis=1)

In [119]:
employment_map = {'govt':0,'private':1,'self_employed':2,'without_pay':3}

df_drop_copy['employment_type'] = df_drop_copy['employment_type'].map(employment_map)


In [120]:
df_drop_copy['occupation'].value_counts()

 Prof-specialty       4038
 Craft-repair         4030
 Exec-managerial      3992
 Adm-clerical         3721
 Sales                3584
 Other-service        3212
 Machine-op-inspct    1966
 Transport-moving     1572
 Handlers-cleaners    1350
 Farming-fishing       989
 Tech-support          912
 Protective-serv       644
 Priv-house-serv       143
 Armed-Forces            9
Name: occupation, dtype: int64

In [121]:
df_drop_copy[['occupation','classification','work class']].groupby(['work class','occupation']).mean()

classification
work class        occupation                        
 Federal-gov       Adm-clerical             0.680380
                   Armed-Forces             0.888889
                   Craft-repair             0.666667
                   Exec-managerial          0.486034
                   Farming-fishing          0.750000
                   Handlers-cleaners        0.909091
                   Machine-op-inspct        0.857143
                   Other-service            0.911765
                   Prof-specialty           0.467066
                   Protective-serv          0.481481
                   Sales                    0.642857
                   Tech-support             0.621212
                   Transport-moving         0.666667
 Local-gov         Adm-clerical             0.882562
                   Craft-repair             0.734266
                   Exec-managerial          0.523585
                   Farming-fishing          0.931034
                   Handlers-cleaners        0.847826
                   Machine-op-inspct        0.818182
                   Other-service            0.941799
                   Prof-specialty           0.638728
                   Protective-serv          0.555921
                   Sales                    0.571429
                   Tech-support             0.605263
                   Transport-moving         0.895652
 Private           Adm-clerical             0.887934
                   Craft-repair             0.775270
                   Exec-managerial          0.517567
                   Farming-fishing          0.933333
                   Handlers-cleaners        0.944223
...                                              ...
 Self-emp-not-inc  Craft-repair             0.822180
                   Exec-managerial          0.631854
                   Farming-fishing          0.851163
                   Handlers-cleaners        0.800000
                   Machine-op-inspct        0.685714
                   Other-service            0.930636
                   Prof-specialty           0.435616
                   Protective-serv          0.833333
                   Sales                    0.662234
                   Tech-support             0.576923
                   Transport-moving         0.754237
 State-gov         Adm-clerical             0.892430
                   Craft-repair             0.745455
                   Exec-managerial          0.569892
                   Farming-fishing          0.866667
                   Handlers-cleaners        0.888889
                   Machine-op-inspct        0.615385
                   Other-service            0.967480
                   Prof-specialty           0.593052
                   Protective-serv          0.750000
                   Sales                    0.727273
                   Tech-support             0.857143
                   Transport-moving         0.829268
 Without-pay       Adm-clerical             1.000000
                   Craft-repair             1.000000
                   Farming-fishing          1.000000
                   Handlers-cleaners        1.000000
                   Machine-op-inspct        1.000000
                   Other-service            1.000000
                   Transport-moving         1.000000

[81 rows x 1 columns]

In [122]:
df_drop_copy.drop(labels=['work class','education','occupation'],axis=1,inplace=True)
df_drop_copy.head(10)

,age,fnlwgt,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per -week,country,classification,employment_type
0,39,77516,13,1,3,0,1,2174,0,40,United-States,1,0
1,50,83311,13,0,2,0,1,0,0,13,United-States,1,2
2,38,215646,9,1,3,0,1,0,0,40,United-States,1,1
3,53,234721,7,0,2,3,1,0,0,40,United-States,1,1
4,28,338409,13,0,1,3,0,0,0,40,Cuba,1,1
5,37,284582,14,0,1,0,0,0,0,40,United-States,1,1
6,49,160187,5,1,3,3,0,0,0,16,Jamaica,1,1
7,52,209642,9,0,2,0,1,0,0,45,United-States,0,2
8,31,45781,14,1,3,0,0,14084,0,50,United-States,0,1
9,42,159449,13,0,2,0,1,5178,0,40,United-States,0,1


In [123]:
df_drop_copy['country'].value_counts()

 United-States                 27504
 Mexico                          610
 Philippines                     188
 Germany                         128
 Puerto-Rico                     109
 Canada                          107
 India                           100
 El-Salvador                     100
 Cuba                             92
 England                          86
 Jamaica                          80
 South                            71
 Italy                            68
 China                            68
 Dominican-Republic               67
 Vietnam                          64
 Guatemala                        63
 Japan                            59
 Columbia                         56
 Poland                           56
 Haiti                            42
 Iran                             42
 Taiwan                           42
 Portugal                         34
 Nicaragua                        33
 Peru                             30
 Greece                           29
 

In [133]:
def country_converter(x):
    if x['country']==' United-States':
        return 'US'
    else:
        return 'Non-US'

In [134]:
df_drop_copy['country_code']=df_drop_copy.apply(country_converter, axis=1)

In [139]:
df_drop_copy.head()

,age,fnlwgt,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per -week,classification,employment_type,country_code
0,39,77516,13,1,3,0,1,2174,0,40,1,0,0
1,50,83311,13,0,2,0,1,0,0,13,1,2,0
2,38,215646,9,1,3,0,1,0,0,40,1,1,0
3,53,234721,7,0,2,3,1,0,0,40,1,1,0
4,28,338409,13,0,1,3,0,0,0,40,1,1,1


In [136]:
country_map = {'US':0,'Non-US':1}

df_drop_copy['country_code'] = df_drop_copy['country_code'].map(country_map)

In [138]:
df_drop_copy.drop('country',axis=1,inplace=True)

In [140]:
df_drop_copy['classification'].value_counts()

1    22654
0     7508
Name: classification, dtype: int64

In [148]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [142]:
X=df_drop_copy.drop('classification',axis=1)
y=df_drop_copy['classification']

In [150]:
X[X.columns]=sc_X.fit_transform(X[X.columns])

D:\Softwares\Python\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Softwares\Python\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [151]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2, stratify=y)

In [152]:
from imblearn.over_sampling import SMOTE
os=SMOTE(random_state=0)
columns=X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train,y_train)
os_data_X=pd.DataFrame(data=os_data_X,columns=columns)
os_data_y=pd.DataFrame(data=os_data_y,columns=['y'])

In [153]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(os_data_X, os_data_y)

D:\Softwares\Python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [154]:
print("Accuracy ", classifier.score(X_test, y_test)*100)

Accuracy  81.38571191778551


In [155]:
print(classification_report(y_test,classifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.64      0.58      0.61      1502
           1       0.87      0.89      0.88      4531

   micro avg       0.81      0.81      0.81      6033
   macro avg       0.75      0.74      0.74      6033
weighted avg       0.81      0.81      0.81      6033

